In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
# from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import datetime

In [21]:
s_data = pd.read_csv('data/samsung.csv', encoding = 'euc-kr')
s_data.head(5)

,일자,시가,고가,저가,종가,거래량
0,2020-01-31,"57,800","58,400","56,400","56,400","19,749,457"
1,2020-01-30,"58,800","58,800","56,800","57,200","20,821,939"
2,2020-01-29,"59,100","59,700","58,800","59,100","16,446,102"
3,2020-01-28,"59,400","59,400","58,300","58,800","23,664,541"
4,2020-01-23,"61,800","61,800","60,700","60,800","14,916,555"


In [22]:
s_data = s_data.rename(columns = {'일자' : 'datetime', '시가' : 'time_price', '고가' : 'high_price', '저가':'row_price', '종가':'final_price','거래량':'amount'})

In [23]:
s_data.head(5)

,datetime,time_price,high_price,row_price,final_price,amount
0,2020-01-31,"57,800","58,400","56,400","56,400","19,749,457"
1,2020-01-30,"58,800","58,800","56,800","57,200","20,821,939"
2,2020-01-29,"59,100","59,700","58,800","59,100","16,446,102"
3,2020-01-28,"59,400","59,400","58,300","58,800","23,664,541"
4,2020-01-23,"61,800","61,800","60,700","60,800","14,916,555"


In [24]:
s_data = s_data.replace('\D', '', regex=True).astype(int)

In [25]:
s_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426 entries, 0 to 425
Data columns (total 6 columns):
datetime       426 non-null int32
time_price     426 non-null int32
high_price     426 non-null int32
row_price      426 non-null int32
final_price    426 non-null int32
amount         426 non-null int32
dtypes: int32(6)
memory usage: 10.1 KB


In [26]:
s_data.head(5)

,datetime,time_price,high_price,row_price,final_price,amount
0,20200131,57800,58400,56400,56400,19749457
1,20200130,58800,58800,56800,57200,20821939
2,20200129,59100,59700,58800,59100,16446102
3,20200128,59400,59400,58300,58800,23664541
4,20200123,61800,61800,60700,60800,14916555


In [27]:
s_data_drop = s_data.copy()
s_data_drop = s_data_drop.drop(['final_price'], axis = 1)

In [28]:
s_data = s_data.values[0:].astype(np.float)
s_data_drop = s_data_drop.values[0:].astype(np.float)

In [29]:
s_data

array([[20200131.,    57800.,    58400.,    56400.,    56400., 19749457.],
       [20200130.,    58800.,    58800.,    56800.,    57200., 20821939.],
       [20200129.,    59100.,    59700.,    58800.,    59100., 16446102.],
       ...,
       [20180509.,    52600.,    52800.,    50900.,    50900., 16128305.],
       [20180508.,    52600.,    53200.,    51900.,    52600., 23104720.],
       [20180504.,    53000.,    53900.,    51800.,    51900., 39565391.]])

In [30]:
s_data_drop

array([[20200131.,    57800.,    58400.,    56400., 19749457.],
       [20200130.,    58800.,    58800.,    56800., 20821939.],
       [20200129.,    59100.,    59700.,    58800., 16446102.],
       ...,
       [20180509.,    52600.,    52800.,    50900., 16128305.],
       [20180508.,    52600.,    53200.,    51900., 23104720.],
       [20180504.,    53000.,    53900.,    51800., 39565391.]])

In [31]:
print (s_data_drop.shape)

x = s_data_drop
y = s_data[:, [4]]

(426, 5)


In [32]:
data_x = []
data_y = []

for i in range(0, len(y) - 5):
    _x = x[i : i+5]
    _y = y[i + 5] # 다음 나타날 주가(정답)
    if i is 0:
        print(_x, "->", _y) # 첫번째 행만 출력해 봄
    data_x.append(_x) # dataX 리스트에 추가
    data_y.append(_y) # dataY 리스트에 추가

[[20200131.    57800.    58400.    56400. 19749457.]
 [20200130.    58800.    58800.    56800. 20821939.]
 [20200129.    59100.    59700.    58800. 16446102.]
 [20200128.    59400.    59400.    58300. 23664541.]
 [20200123.    61800.    61800.    60700. 14916555.]] -> [62300.]


In [33]:
train_size = int(len(data_y) * 0.8)
test_size = len(data_y) - train_size

train_x = np.array(data_x[0:train_size])
train_y = np.array(data_y[0:train_size])

test_x = np.array(data_x[train_size:len(data_x)])
test_y = np.array(data_y[train_size:len(data_y)])

In [34]:
print(train_x.shape)
print (test_x.shape)

(336, 5, 5)
(85, 5, 5)


In [35]:
train_x = train_x.reshape(336, 25, 1)
test_x = test_x.reshape(85, 25, 1)

In [36]:
print(train_x.shape)
print (test_x.shape)

(336, 25, 1)
(85, 25, 1)


In [37]:
model = Sequential()
model.add(LSTM(10, activation = 'relu', input_shape = (25,1), return_sequences = True))   # (열, 몇 개씩 자를지)
model.add(LSTM(5, activation = 'relu', return_sequences = True))
model.add(LSTM(6, activation = 'relu', return_sequences = True))
model.add(LSTM(9, activation = 'relu', return_sequences = True))
model.add(LSTM(1, activation = 'relu', return_sequences = False))

In [38]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])  
model.fit(train_x, train_y, epochs=100, batch_size=1, verbose =1)  
loss, mse = model.evaluate(test_x, test_y, batch_size=1)
print("loss:",loss,"mse:", mse) 


Epoch 1/100
336/336 [==============================] - 13s 40ms/step - loss: 2167935530.6667 - mse: 2167935744.0000
Epoch 2/100
336/336 [==============================] - 12s 35ms/step - loss: 2167935530.6667 - mse: 2167937024.0000
Epoch 3/100
336/336 [==============================] - 12s 35ms/step - loss: 2167935530.6667 - mse: 2167936256.0000
Epoch 4/100
336/336 [==============================] - 12s 35ms/step - loss: 2167935530.6667 - mse: 2167935232.0000
Epoch 5/100
336/336 [==============================] - 12s 35ms/step - loss: 2167935530.6667 - mse: 2167935232.0000
Epoch 6/100
336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167935232.0000
Epoch 7/100
336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167935488.0000
Epoch 8/100
336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167936000.0000
Epoch 9/100
336/336 [==============================] - 13s 37ms/step - l

336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167935488.0000
Epoch 71/100
336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167936000.0000
Epoch 72/100
336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167935232.0000
Epoch 73/100
336/336 [==============================] - 12s 37ms/step - loss: 2167935530.6667 - mse: 2167935488.0000
Epoch 74/100
336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167935744.0000
Epoch 75/100
336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167936256.0000
Epoch 76/100
336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167935488.0000
Epoch 77/100
336/336 [==============================] - 12s 36ms/step - loss: 2167935530.6667 - mse: 2167936512.0000
Epoch 78/100
336/336 [==============================] - 12s 36ms/step - loss:

In [ ]:
y_predict = model.predict(test_x, batch_size=1)

# RMSE 구하기
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))
print('RMSE :', RMSE(test_y,y_predict))

(336, 25, 1)


In [22]:
# s_final = s_data[['final_price']]
# s_final.head(5)

In [23]:
# s_data = s_data.drop(['final_price'], axis = 1)

In [24]:
# x.datetime = x.datetime.apply(pd.to_datetime)

# x['year'] = x['datetime'].dt.year
# x['month'] = x['datetime'].dt.month
# x['day'] = x['datetime'].dt.day

# del x['datetime']

In [25]:
# x = x.apply(pd.to_numeric, errors='coerce')
# y = y.apply(pd.to_numeric, errors='coerce')

In [26]:
s_data.head(5)

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [ ]:
s_data.info()

In [ ]:
print(s_data.shape)

In [ ]:
# date = s_data[['datetime']]
# time = s_data[['time_price']]
# high = s_data[['high_price']]
# row = s_data[['row_price']]
# amount = s_data[['amount']]


In [ ]:
price = s_data.values[1:].astype(np.float)
volume = s_final.values[1:].astype(np.float)

In [ ]:
price = price[:,:-1]
volume = s_final[:,-1:]

x = np.concatenate((price, volume), axis=1) # axis=1, 세로로 합친다

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data, final, test_size=0.2, shuffle = False)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

In [ ]:
# DNN
model = Sequential()
model.add(Dense(5, input_dim =5))
model.add(Dense(10))
model.add(Dense(12))
model.add(Dense(10))
model.add(Dense(5))
model.add(Dense(1))

In [ ]:
x_train = x_train.values
y_train = y_train.values
x_test = x_test.values
y_test = y_test.values

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 5, 1)
x_test = x_test.reshape(x_test.shape[0], 5, 1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
model = Sequential()
model.add(Dense(5, input_dim =5))
model.add(Dense(10))
model.add(Dense(12))
model.add(Dense(10))
model.add(Dense(5))
model.add(Dense(1))

In [ ]:
model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])
model.fit(x_train, y_train, validation_split = 0.2, epochs=100, batch_size=1)


In [ ]:
mse, acc = model.evaluate(x_test, y_test, batch_size=1)
print ('loss:', loss, 'mse:', mse)